# Тестовое задание:

требуется разработать модель, которая будет способна различать заголовки реальных и выдуманных новостей.

Для обучения модели используйте данные из файла `train.tsv`. В файле находится таблица, состоящая из двух колонок. 
В колонке title записан заголовок новости. В колонке is_fake содержатся метки: 0 – новость реальная; 1 – новость выдуманная.
Для демонстрации работы модели используйте данные тестового набора из файла `test.tsv`. В нем также есть колонка title, данные которой являются входными для вашей модели.

Вам нужно скопировать файл `test.tsv`, переименовать его в `predictions.tsv` и заполнить колонку is_fake значениями предсказаний вашей модели, аналогично `train.tsv`. 
Изначально колонка заполнена значением 0.

---

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_data = pd.read_csv('dataset/train.tsv', sep = '\t')
train_data.head(10)

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1
5,Российским студентам запретят учиться за рубежом,1
6,Путин пишет книгу об истории Украины,1
7,Россияне обхитрили рост цен,0
8,Звезда «Ворониных» раскрыл подробности о своем...,0
9,Microsoft объявила дату выхода очков дополненн...,0


In [16]:
# разделим тренировочный датасет на train/test - нам же нужно будет как-то оценить работоспособность модели

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data['title'], train_data['is_fake'], 
                                                    test_size = 0.33, random_state = 42)
X_train[:10]

2788          Роскосмос опроверг отказ от полетов на Луну
5480    Ученые считают найденную на руинах Вавилона та...
4210      Чистая прибыль АСВ перекрыла убытки за 2008 год
2399          Samsung показал шлем виртуальной реальности
5625    Для спасения от малярии медики посоветовали сп...
650     Несовершеннолетним запретят пользоваться интер...
4602           Звезду «Такси» Сами Насери избили в Москве
708     Строительная компания Strabag отложила IPO на ...
3483    Французские рабочие вновь захватили в заложник...
5306    Физики анонсировали создание «микроба Шредингера»
Name: title, dtype: object

# 1. попробуем подход: TF-IDF:

In [17]:
# Токенизируем входные данные и строим словарь всех известных слов

from sklearn.svm import SVC 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

vectorizer = TfidfVectorizer()
train_x_vect = vectorizer.fit_transform(X_train)
#print(train_x_vect)

In [18]:
train_x_vect.toarray().shape

#по всем 3857 сообщениям набралось 14318 уникальных слов

(3857, 14318)

In [19]:
#попробуем метод опорных векоторов с линейным ядром - выберем наиболее оптимальный параметр перебором

grid = {'C': np.power(10.0, np.arange(-5, 6))}

cv = KFold(n_splits = 5, shuffle = True, random_state = 42)
clf = SVC(kernel = 'linear', random_state = 42)

gs = GridSearchCV(clf, grid, scoring = 'accuracy', cv = cv)
gs.fit(train_x_vect, y_train)

print(gs.best_score_)
print(gs.best_params_)

0.818251984166986
{'C': 1.0}


In [20]:
clf = SVC(kernel = 'linear', random_state = 42, C = gs.best_params_['C'])
clf.fit(train_x_vect, y_train)
#clf.coef_.toarray()

SVC(kernel='linear', random_state=42)

In [21]:
#последние 20 самых важных для моедли слов - можно также поработать над удалением стоп слов.

A = clf.coef_.indices
B = np.abs(clf.coef_.data)

Words = A[B.argsort()[-20:][::-1]]
for i in Words:
    print(vectorizer.get_feature_names()[i])

запретят
навального
что
рпц
лукашенко
суд
нефти
при
сколково
госдуме
коронавируса
гибдд
для
кпрф
дмитрий
нефть
честь
процентов
чтобы
нато


In [26]:
#проверяем на тестовых данных

from sklearn.metrics import f1_score

y_pred = clf.predict(vectorizer.transform(X_test))
f1_score(y_test, y_pred, average = 'binary')

0.8235887096774194